In [ ]:
%pip install -U torch transformers python-dotenv accelerate huggingface_hub bitsandbytes peft trl wandb -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import os
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import Dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split
from datetime import datetime
import wandb
from google.colab import userdata

In [ ]:
from src.model import ModelLoader
from src.utils import make_prompt, make_samples_from_df, plot_confusion_matrix
from src.pipeline import TextGenerationPipeline
from src.dataset import MyDataset

In [ ]:
from huggingface_hub import login
login(token=userdata.get('HUGGINGFACE_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Loading and processing the dataset

In [ ]:
splits = {'train': 'data/train-00000-of-00001-7b34565378f02992.parquet', 'val': 'data/val-00000-of-00001-d7338c59b5e5031f.parquet', 'test': 'data/test-00000-of-00001-c830a979da438bff.parquet'}
df_train = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["val"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df_train.head()

,Text,Label,dataset,__index_level_0__
0,¿Por qué hacer que mejore? ¿Por qué la escuela...,1,Hackaton Somos NLP,5071
1,- Roma \n- Milán \n- Nápoles \n- Turín \n- Pal...,0,Hackaton Somos NLP,9391
2,¿Cuál es la noticia más importante en tu área ...,0,Hackaton Somos NLP,1752
3,Una semana de identificación izquierda más bie...,1,Hackaton Somos NLP,5363
4,Podría ser que no estudiaras de la manera corr...,0,Hackaton Somos NLP,107


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136136 entries, 0 to 136135
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Text               136136 non-null  object
 1   Label              136136 non-null  int64 
 2   dataset            136136 non-null  object
 3   __index_level_0__  136136 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 4.2+ MB


In [ ]:
df_train.Label.value_counts()

,count
Label,
0,75496
1,60640


In [ ]:
# Filtra los datos con Label igual a 1
df_label_1 = df_train[df_train['Label'] == 1]

# Filtra los datos con Label igual a 0
df_label_0 = df_train[df_train['Label'] == 0]

# Toma 10,000 muestras aleatorias de cada grupo
sampled_label_1 = df_label_1.sample(n=10000, random_state=42)
sampled_label_0 = df_label_0.sample(n=10000, random_state=42)

# Combina ambos DataFrames
sampled_df = pd.concat([sampled_label_1, sampled_label_0])

# Reordena las filas aleatoriamente

df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
df.Label.value_counts()

,count
Label,
0,10000
1,10000


In [ ]:
# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
            Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.
texto: {data_point["Text"]}
etiqueta: {data_point["Label"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.
texto: {data_point["Text"]}
etiqueta: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'Label']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["Text"])

<ipython-input-14-9030f6d2d232>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
<ipython-input-14-9030f6d2d232>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [ ]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [ ]:
train_data['text'][3]

'Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.\ntexto: Cada vez que pienso en cómo eran las cosas antes de que naciera me da nostalgia no tenía ninguna neurona que me dijera cómo soy responsable de todas las cosas malas que están mal en este mundo y puedo arreglarlo si me deshago de la enfermedad que soy.\netiqueta: 1'

## Loading the model and tokenizer ( in 4-bit quantization to save the GPU memory)

In [ ]:
base_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## tracking experiment with wandb

In [ ]:
today = datetime.today()
run_date = today.strftime("%d-%m-%Y_%H_%M")

wb_token = userdata.get('wandb')

wandb.login(key=wb_token)
run = wandb.init(
    project='zero and few shot llama-3.1-8b-it on suicide intent-spanish',
    job_type="training",
    name=f"zero_and_few_shot_llama_{run_date}",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Testing model

In [ ]:
def evaluate(y_true, y_pred):
    labels = ["1", "0"]
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    # Get classification report as a dictionary
    class_report_dict = classification_report(y_true, y_pred, target_names=labels, labels=list(range(len(labels))), output_dict=True)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

    return accuracy, class_report_dict, conf_matrix,labels

def predict(test, model, tokenizer):
    y_pred = []
    categories = ["1", "0"]

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["Text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2,
                        temperature=0.1)

        result = pipe(prompt)
        answer = result[0]['generated_text'].split("etiqueta:")[-1].strip()

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred

In [ ]:
y_pred = predict(X_test, model, tokenizer)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
y_true = y_true.astype(str)

In [ ]:
accuracy, class_report, conf_matrix, labels = evaluate(y_true, y_pred)

Accuracy: 0.720
Accuracy for label 1: 0.750
Accuracy for label 0: 0.688

Classification Report:
              precision    recall  f1-score   support

           1       0.71      0.75      0.73      1006
           0       0.73      0.69      0.71       994

    accuracy                           0.72      2000
   macro avg       0.72      0.72      0.72      2000
weighted avg       0.72      0.72      0.72      2000


Confusion Matrix:
[[755 251]
 [310 684]]


In [ ]:
wandb.log({"Accuracy": accuracy})
wandb.log({"classification_report": class_report})

In [ ]:
# Create a mapping of class names to integer indices
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Convert y_true and y_pred from strings to indices
y_true_int = [label_to_index[label] for label in y_true]
y_pred_int = [label_to_index[label] for label in y_pred]

# Log confusion matrix with the updated integer values
wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(probs=None, y_true=y_true_int, preds=y_pred_int, class_names=labels)})

In [ ]:
wandb.finish()

Accuracy,▁
Accuracy,0.7195


# few shot

## Few-Shot Learning:
create samples for test metrics the model

In [ ]:
samples = make_samples_from_df(df_val, n_samples = 5)
print(samples)

"texto: La muerte parece amable, la soledad realmente me está lastimando. Nadie me echará de menos cuando me haya ido" : "etiqueta: 1",
"texto: te quiero al final de mi vida
Quiero ver tu cara cuando caiga con gracia
en el momento en que muero
está eso bien" : "etiqueta: 1",
"texto: mi MC está actualmente encarcelado y en una máquina de muerte; ¿Te imaginas lo que pasaría si tuviera una mascota? xdd" : "etiqueta: 1",
"texto: Tengo 21 años y nunca he tenido una relación adicta al porno, a los juegos y a los abucheos. He estado tratando de cambiar durante los últimos 2-3 años, pero siempre he fallado. Odio mi trabajo en todo lo que puedo pensar ahora es en cómo me voy a suicidar." : "etiqueta: 1",
"texto: No tengo hogar, tengo una casa en la que vivo, pero no la siento como un hogar cuando estoy en el trabajo. Constantemente pienso que quiero ir a casa y luego, cuando llego a la casa, todavía me encuentro pensando que realmente Quiero ir a casa, no sé dónde está, aunque a veces salgo por

In [ ]:
text_in = X_test["Text"].iloc[0]
text_in

'Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.\ntexto: Estoy listo para morir.2 años de tiempo prestado. Ahora estoy listo\netiqueta:'

In [ ]:
def make_prompt(samples: str, text: str) -> str:
    """
    function to generate prompt
    """
    prompt = f"""Basado en los ejemplos: {samples}. {text}""".strip()
    return prompt

In [ ]:
few_shot_prompt = make_prompt(samples, text_in)
print(few_shot_prompt)

Basado en los ejemplos: "texto: La muerte parece amable, la soledad realmente me está lastimando. Nadie me echará de menos cuando me haya ido" : "etiqueta: 1",
"texto: te quiero al final de mi vida
Quiero ver tu cara cuando caiga con gracia
en el momento en que muero
está eso bien" : "etiqueta: 1",
"texto: mi MC está actualmente encarcelado y en una máquina de muerte; ¿Te imaginas lo que pasaría si tuviera una mascota? xdd" : "etiqueta: 1",
"texto: Tengo 21 años y nunca he tenido una relación adicta al porno, a los juegos y a los abucheos. He estado tratando de cambiar durante los últimos 2-3 años, pero siempre he fallado. Odio mi trabajo en todo lo que puedo pensar ahora es en cómo me voy a suicidar." : "etiqueta: 1",
"texto: No tengo hogar, tengo una casa en la que vivo, pero no la siento como un hogar cuando estoy en el trabajo. Constantemente pienso que quiero ir a casa y luego, cuando llego a la casa, todavía me encuentro pensando que realmente Quiero ir a casa, no sé dónde está, 

In [ ]:
def predict_with_few_shot(test, n_samples, model, tokenizer):
    y_pred = []
    categories = ["1", "0"]
    samples = make_samples_from_df(df_val, n_samples = n_samples)
    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["Text"]
        few_shot_prompt = make_prompt(samples, prompt)
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2,
                        temperature=0.1)

        result = pipe(few_shot_prompt)
        answer = result[0]['generated_text'].split("etiqueta: ")[-1].strip()

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred,samples

In [ ]:
today = datetime.today()
run_date = today.strftime("%d-%m-%Y_%H_%M")

wb_token = userdata.get('wandb')

wandb.login(key=wb_token)
run = wandb.init(
    project='zero and few shot llama-3.1-8b-it on suicide intent-spanish',
    job_type="training",
    name=f"zero_and_few_shot_llama_{run_date}",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
n_samples = 30

In [ ]:
y_pred,samples = predict_with_few_shot(X_test,n_samples, model, tokenizer)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
accuracy, class_report, conf_matrix, labels = evaluate(y_true, y_pred)

Accuracy: 0.763
Accuracy for label 1: 0.684
Accuracy for label 0: 0.844

Classification Report:
              precision    recall  f1-score   support

           1       0.82      0.68      0.74      1006
           0       0.73      0.84      0.78       994

    accuracy                           0.76      2000
   macro avg       0.77      0.76      0.76      2000
weighted avg       0.77      0.76      0.76      2000


Confusion Matrix:
[[688 318]
 [155 839]]


In [ ]:
wandb.log({"samples": samples, "n_samples" : n_samples})

In [ ]:
wandb.log({"Accuracy": accuracy})
wandb.log({"classification_report": class_report})

In [ ]:
wandb.finish()

Accuracy,▁
n_samples,▁
Accuracy,0.7635
n_samples,30
samples,"""texto: La muerte pa..."
